# Rendre inactifs 6264 acteurs reparation LVAO (ancien CMA)

[Ticket](https://www.notion.so/accelerateur-transition-ecologique-ademe/Supprimer-les-artisans-Ferm-s-de-la-vieille-source-CMA-non-r-paracteurs-12e6523d57d78045a70ed85ad8a47796)

 - **QUOI**: modifier 6264 acteurs LVAO `_reparation_` (anciennement CMA) en changeant `statut` = `INACTIF`
 - **POURQUOI**: les entrées dans la base sont obsolètes
 - **COMMENT**:
    - intégration avec la base SIRENE en local
    - génération d'un export et revue manuelle
    - génération des requettes SQL sur la base de l'export



## Paramètres

In [13]:
import os
from datetime import datetime

# Base de données
DB_TEST_URL = os.environ["DB_TEST_URL"]
DB_PROD_URL = os.environ["DB_PROD_URL"]
DB_TABLE = "qfdmo_acteur"
DB_STATUS_FIELD = "statut"
DB_STATUS_VALUE = "INACTIF"
DB_COMMENT_FIELD = "commentaires"
DB_COMMENT_VALUE = "20241031_acteurs_inactifs_reparation_lvao_ancien_cma"
DB_ID_FIELD = "identifiant_unique"
DB_TIMESTAMP_FIELD = "modifie_le"
# on ne touche pas à se qui a été modifié end prod
# depuis l'export
DB_TIMESTAMP_SAFEGUARD = datetime.strptime("2024-10-29 00:00:00", "%Y-%m-%d %H:%M:%S")

# File system
#DIR_CURRENT = os.path.dirname(os.path.abspath(__file__))
#DIR_DATA = os.path.join(DIR_CURRENT, "data")

# Export validé
DATA_FILEPATH = "./data/6264_acteurs_a_desactiver.csv"
DATA_FILTER_KEY = "validation"
# note: seule un échantillion à été validé donc on garde
# tout sauf 1 entrée modifiée sur prod entre temps
DATA_FILTER_EXCLUDE = "NE PAS TOUCHER"
DATA_FILTER_LENGTH = 6264 # on s'attend à modifier 6264 acteurs
DATA_STATUS_FIELD = "statut"
DATA_STATUS_CURRENT = "ACTIF"

# Les champs de la source de données SIRENE nous indiquant
# que soit l'entreprise est en cessation (C), soit
# l'établissement est fermé (F)
DATA_SIREN_STATUS_FIELD = "etatAdministratifUniteLegale"
DATA_SIREN_STATUS_VALUE = "C"
DATA_SIRET_STATUS_FIELD = "etatAdministratifEtablissement"
DATA_SIRET_STATUS_VALUE = "F"


## Import des librairies

In [14]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from rich import print
from rich.progress import track

## Valider connection à la base de données

In [15]:
# Create and test engines
#assert DB_TEST_URL != DB_PROD_URL, "DB_TEST_URL et DB_PROD_URL doivent être différents"
db_test_engine = create_engine(DB_TEST_URL)
db_prod_engine = create_engine(DB_PROD_URL)
for engine in [db_test_engine, db_prod_engine]:
    conn = engine.connect()
    conn.execute(f"SELECT * FROM {DB_TABLE} LIMIT 1")
    conn.close()

## Lecture et validation de la data

In [16]:
df = pd.read_csv(DATA_FILEPATH).replace({np.nan: None})
df.head()

,validation,django,sirene_unite_legale,sirene_etablissement,etatAdministratifUniteLegale,etatAdministratifEtablissement,nom,description,siret,identifiant_unique,adresse,adresse_complement,code_postal,ville,statut
0,None,lien,lien,lien,A,F,Axelle Bijoux,None,80458761600025,axelle_bijoux_156582_reparation_0689109752,7 RUE DU SALLE,None,29000,QUIMPER,ACTIF
1,None,lien,lien,lien,C,F,Carre Confidentiel,None,75372533200021,carre_confidentiel_168156_reparation_0648090745,27 AVENUE THIERS,ETG 1,33100,BORDEAUX,ACTIF
2,None,lien,lien,lien,C,F,Chez Le Geek,None,88491920000011,chez_le_geek_114232_reparation_0658534768,13 RUE SAINT PIERRE,None,42600,MONTBRISON,ACTIF
3,None,lien,lien,lien,A,F,De Fil En Aiguille,None,41292816000024,de_fil_en_aiguille_177625_reparation_0643053932,19 RUE CORNUE,None,84000,AVIGNON,ACTIF
4,None,lien,lien,lien,A,F,Diapason Horlogerie,None,52865747100025,diapason_horlogerie_166744_reparation_0609920110,01 RUE RENAU D'ELISSAGARAY,None,64500,ST JEAN DE LUZ,ACTIF


In [17]:
# On filtre les acteurs à modifier
filter_validation = df[DATA_FILTER_KEY] != DATA_FILTER_EXCLUDE
filter_siren = df[DATA_SIREN_STATUS_FIELD] == DATA_SIREN_STATUS_VALUE
filter_siret = df[DATA_SIRET_STATUS_FIELD] == DATA_SIRET_STATUS_VALUE

df = df[filter_validation & (filter_siren | filter_siret)]
print(f"Nombre d'acteurs à modifier: {len(df)}")
assert len(df) == DATA_FILTER_LENGTH, f"On devrait avoir {DATA_FILTER_LENGTH} acteurs à modifier"

Nombre d'acteurs à modifier: 6264

In [18]:
# On s'assure qu'ils étaient tous actifs
assert all(df[DATA_STATUS_FIELD] == DATA_STATUS_CURRENT), "Tous les acteurs devraient être actifs"
# Qu'on avait pas de doublon sur le champ identifiant
assert len(df) == len(df[DB_ID_FIELD].unique()), f"Présence de doublons sur le champ {DB_ID_FIELD}"

## Logique de modification de la base

In [26]:
def df_update_db(df: pd.DataFrame, engine):
    verbose = True
    conn = engine.connect()
    with conn.begin():
        for i, row in track(df.iterrows(), total=len(df)):
            # Avant la modification on s'assure que l'entrée
            # n'a pas été modifiée depuis la sauvegarde
            if verbose:
                print("MAJ de l'acteur:", row[DB_ID_FIELD])
            query_check = f"SELECT {DB_TIMESTAMP_FIELD} FROM {DB_TABLE} WHERE {DB_ID_FIELD} = '{row[DB_ID_FIELD]}'"
            last_modified = conn.execute(query_check).fetchone()[0].replace(tzinfo=None)
            if verbose:
                print(f"\t{DB_TIMESTAMP_FIELD} = ", last_modified,"vs. safeguard =",str(DB_TIMESTAMP_SAFEGUARD))
            # parse timestamps and ensure last_modified < DB_TIMESTAMP_SAFEGUARD
            assert last_modified < DB_TIMESTAMP_SAFEGUARD, f"La date de dernière modification de l'acteur {row[DB_ID_FIELD]} est postérieure à la date de sauvegarde"

            # On met à jour le statut et les commentaires
            query_update = f"""
                UPDATE {DB_TABLE}
                SET
                    {DB_STATUS_FIELD} = '{DB_STATUS_VALUE}',
                    {DB_COMMENT_FIELD} = '{DB_COMMENT_VALUE}',
                    -- Au cas où notre champ timestamp n'est pas mis à jour automatiquement par le schema
                    {DB_TIMESTAMP_FIELD} = NOW()
                WHERE {DB_ID_FIELD} = '{row[DB_ID_FIELD]}'"""
            if verbose:
                print(query_update)
            conn.execute(query_update)
            verbose = False
    print(f"{len(df)} acteurs mis à jour")
    conn.close()

## Modification de la base

### Test

In [27]:
df_update_db(df, db_test_engine)

MAJ de l'acteur: axelle_bijoux_156582_reparation_0689109752

modifie_le =  2023-10-31 09:56:51.215125 vs. safeguard = 2024-10-29 00:00:00

UPDATE qfdmo_acteur
                SET
                    statut = 'INACTIF',
                    commentaires = '20241031_acteurs_inactifs_reparation_lvao_ancien_cma',
                    -- Au cas où notre champ timestamp n'est pas mis à jour automatiquement par le schema
                    modifie_le = NOW()
                WHERE identifiant_unique = 'axelle_bijoux_156582_reparation_0689109752'

6264 acteurs mis à jour

## Production

In [25]:
raise Exception("Sur la point de modifier la base de prod, retirer cette ligne pour continuer")

Exception: Sur la point de modifier la base de prod, retirer cette ligne pour continuer

In [ ]:
df_update_db(df, db_prod_engine)